In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.8.1
    Uninstalling duckdb-0.8.1:
      Successfully uninstalled duckdb-0.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.6/317.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 864.6 kB/s eta 0:00:00


"A diferencia de la competencia 1, ahora vamos a usar una base de datos en disco. Todo va a ser más lento, pero se va a poder trabajar sin problemas en máquinas de menor capacidad. La DB va a queda en el archivo dmeyf.db, lo probé en un colab básico y funcionó sin problemas."


In [ ]:
# cd "/home/mabilotas/buckets/b1/datasets"

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:////content/dmeyf.db
# %sql duckdb:///content/dmeyf.db

###obviar estas celdas para jupyter
"Cargamos el dataset, y como tenemos la base en el disco, no debemos cargarlo más, la próxima se puede usar directamente"



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/content/drive/MyDrive/DM/DMEyF/competencia_02.csv.gz')
# from read_csv_auto('competencia_02.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


##Ver la tabla

In [8]:
%%sql
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'competencia_02';

,column_name,data_type
0,numero_de_cliente,BIGINT
1,foto_mes,BIGINT
2,active_quarter,BIGINT
3,cliente_vip,BIGINT
4,internet,BIGINT
...,...,...
150,Visa_mconsumototal,DOUBLE
151,Visa_cconsumos,BIGINT
152,Visa_cadelantosefectivo,BIGINT
153,Visa_mpagominimo,DOUBLE


#Features históricas

In [10]:
# Vamos por unos ejemplos simples. Que pasa si quiero agregar un feature que muestre el valor del periodo anterior?
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_02
limit 10


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,29188883,201901,43,NaN
1,29188883,201902,45,43.0
2,29188883,201903,39,45.0
3,29188883,201904,38,39.0
4,29188883,201905,39,38.0
5,29188883,201906,41,39.0
6,29188883,201907,34,41.0
7,29188883,201908,30,34.0
8,29188883,201909,25,30.0
9,29188883,201910,30,25.0


In [11]:
# Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_02
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,29188883,201901,43,NaN,NaN,NaN
1,29188883,201902,45,43.0,2.0,NaN
2,29188883,201903,39,45.0,-6.0,-4.0
3,29188883,201904,38,39.0,-1.0,-7.0
4,29188883,201905,39,38.0,1.0,0.0
5,29188883,201906,41,39.0,2.0,3.0
6,29188883,201907,34,41.0,-7.0,-5.0
7,29188883,201908,30,34.0,-4.0,-11.0
8,29188883,201909,25,30.0,-5.0,-9.0
9,29188883,201910,30,25.0,5.0,0.0


In [12]:
# Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil en los últimos 6 meses, se puede hacer fácilmente
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_ctrx_quarter
from competencia_02
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,avg_6_ctrx_quarter
0,29188883,201901,43,43.000000
1,29188883,201902,45,44.000000
2,29188883,201903,39,42.333333
3,29188883,201904,38,41.250000
4,29188883,201905,39,40.800000
5,29188883,201906,41,40.833333
6,29188883,201907,34,39.857143
7,29188883,201908,30,38.000000
8,29188883,201909,25,35.142857
9,29188883,201910,30,33.857143


In [13]:
# Para el cliente 29188883 para la foto 201910, se puede chequear que la nueva
#  feature se construye con la siguiente cuenta (30 + 25 + 30 + 34 + 41 + 39 + 38) / 7
# Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si
#  se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis

%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6
  , max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6
  , min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_6,ctrx_quarter_max_6,ctrx_quarter_min_6
0,29188883,201901,43,43.000000,43,43
1,29188883,201902,45,44.000000,45,43
2,29188883,201903,39,42.333333,45,39
3,29188883,201904,38,41.250000,45,38
4,29188883,201905,39,40.800000,45,38
5,29188883,201906,41,40.833333,45,38
6,29188883,201907,34,39.857143,45,34
7,29188883,201908,30,38.000000,45,30
8,29188883,201909,25,35.142857,41,25
9,29188883,201910,30,33.857143,41,25


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates

In [14]:
# Un caso más, que ni me voy a molestar en explicar que significa...
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_slope_6
0,29188883,201901,43,NaN
1,29188883,201902,45,2.000000
2,29188883,201903,39,-2.000000
3,29188883,201904,38,-2.100000
4,29188883,201905,39,-1.500000
5,29188883,201906,41,-0.828571
6,29188883,201907,34,-1.250000
7,29188883,201908,30,-1.857143
8,29188883,201909,25,-2.250000
9,29188883,201910,30,-2.250000


###Variables de las que generaremos nuevas variables

In [15]:
# Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables

campos = ['cdescubierto_preacordado','mcuenta_corriente', 'visa_status','ctrx_quarter', 'mpasivos_margen', 'mautoservicio' , 'ctarjeta_debito_transacciones' , 'mtransferencias_emitidas' , 'ctransferencias_emitidas' , 'mpayroll' , 'cpayroll_trx', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mcomisiones', 'mcaja_ahorro', 'mcuentas_saldo', 'mtarjeta_visa_consumo','mprestamos_personales']
print(campos)
print(len(campos))

['cdescubierto_preacordado', 'mcuenta_corriente', 'visa_status', 'ctrx_quarter', 'mpasivos_margen', 'mautoservicio', 'ctarjeta_debito_transacciones', 'mtransferencias_emitidas', 'ctransferencias_emitidas', 'mpayroll', 'cpayroll_trx', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mcomisiones', 'mcaja_ahorro', 'mcuentas_saldo', 'mtarjeta_visa_consumo', 'mprestamos_personales']
18


In [16]:
#lag, delta lag, slope y media para ventana de 6

len(campos)
nuevos_features = ""
for campo in campos:
  nuevos_features += f", lag({campo}, 1) over ventana_6 as {campo}_lag_1"
  nuevos_features += f", lag({campo}, 2) over ventana_6 as {campo}_lag_2"
  nuevos_features += f", lag({campo}, 3) over ventana_6 as {campo}_lag_3"
  nuevos_features += f", lag({campo}, 4) over ventana_6 as {campo}_lag_4"
  nuevos_features += f", lag({campo}, 5) over ventana_6 as {campo}_lag_5"
  nuevos_features += f", lag({campo}, 6) over ventana_6 as {campo}_lag_6"
  nuevos_features += f", {campo} - lag({campo}, 1) over ventana_6 as {campo}_delta_1_lag"
  nuevos_features += f", {campo} - lag({campo}, 2) over ventana_6 as {campo}_delta_2_lag"
  nuevos_features += f", {campo} - lag({campo}, 3) over ventana_6 as {campo}_delta_3_lag"
  nuevos_features += f", {campo} - lag({campo}, 4) over ventana_6 as {campo}_delta_4_lag"
  nuevos_features += f", {campo} - lag({campo}, 5) over ventana_6 as {campo}_delta_5_lag"
  nuevos_features += f", {campo} - lag({campo}, 6) over ventana_6 as {campo}_delta_6_lag"
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_6 as {campo}_slope_6"
  nuevos_features += f", avg({campo}) over ventana_6 as {campo}_media_6"


print(nuevos_features)

, lag(cdescubierto_preacordado, 1) over ventana_6 as cdescubierto_preacordado_lag_1, lag(cdescubierto_preacordado, 2) over ventana_6 as cdescubierto_preacordado_lag_2, lag(cdescubierto_preacordado, 3) over ventana_6 as cdescubierto_preacordado_lag_3, lag(cdescubierto_preacordado, 4) over ventana_6 as cdescubierto_preacordado_lag_4, lag(cdescubierto_preacordado, 5) over ventana_6 as cdescubierto_preacordado_lag_5, lag(cdescubierto_preacordado, 6) over ventana_6 as cdescubierto_preacordado_lag_6, cdescubierto_preacordado - lag(cdescubierto_preacordado, 1) over ventana_6 as cdescubierto_preacordado_delta_1_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 2) over ventana_6 as cdescubierto_preacordado_delta_2_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 3) over ventana_6 as cdescubierto_preacordado_delta_3_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 4) over ventana_6 as cdescubierto_preacordado_delta_4_lag, cdescubierto_preacordado - lag(cdesc

In [18]:
#slope y media para ventana de 3

len(campos)
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"
  nuevos_features += f", avg({campo}) over ventana_3 as {campo}_media_3"

print(nuevos_features)


, regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_3 as cdescubierto_preacordado_slope_3, avg(cdescubierto_preacordado) over ventana_3 as cdescubierto_preacordado_media_3
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as mcuenta_corriente_slope_3, avg(mcuenta_corriente) over ventana_3 as mcuenta_corriente_media_3
, regr_slope(visa_status, cliente_antiguedad) over ventana_3 as visa_status_slope_3, avg(visa_status) over ventana_3 as visa_status_media_3
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3, avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3, avg(mpasivos_margen) over ventana_3 as mpasivos_margen_media_3
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_3 as mautoservicio_slope_3, avg(mautoservicio) over ventana_3 as mautoservicio_media_3
, regr_slope(ctarjeta_debito_transacciones, cliente_ant

In [20]:
#lag, delta lag, slope y media para ventana de 9

len(campos)
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_9 as {campo}_slope_9"
  nuevos_features += f", avg({campo}) over ventana_9 as {campo}_media_9"

print(nuevos_features)


, regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_9 as cdescubierto_preacordado_slope_9, avg(cdescubierto_preacordado) over ventana_9 as cdescubierto_preacordado_media_9
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_9 as mcuenta_corriente_slope_9, avg(mcuenta_corriente) over ventana_9 as mcuenta_corriente_media_9
, regr_slope(visa_status, cliente_antiguedad) over ventana_9 as visa_status_slope_9, avg(visa_status) over ventana_9 as visa_status_media_9
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_9 as ctrx_quarter_slope_9, avg(ctrx_quarter) over ventana_9 as ctrx_quarter_media_9
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_9 as mpasivos_margen_slope_9, avg(mpasivos_margen) over ventana_9 as mpasivos_margen_media_9
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_9 as mautoservicio_slope_9, avg(mautoservicio) over ventana_9 as mautoservicio_media_9
, regr_slope(ctarjeta_debito_transacciones, cliente_ant

el resultado anterior lo uso para esta nueva query

In [ ]:
%%time
%%sql
create or replace table competencia_02 as
SELECT
  *
, lag(cdescubierto_preacordado, 1) over ventana_6 as cdescubierto_preacordado_lag_1, lag(cdescubierto_preacordado, 2) over ventana_6 as cdescubierto_preacordado_lag_2, lag(cdescubierto_preacordado, 3) over ventana_6 as cdescubierto_preacordado_lag_3, lag(cdescubierto_preacordado, 4) over ventana_6 as cdescubierto_preacordado_lag_4, lag(cdescubierto_preacordado, 5) over ventana_6 as cdescubierto_preacordado_lag_5, lag(cdescubierto_preacordado, 6) over ventana_6 as cdescubierto_preacordado_lag_6, cdescubierto_preacordado - lag(cdescubierto_preacordado, 1) over ventana_6 as cdescubierto_preacordado_delta_1_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 2) over ventana_6 as cdescubierto_preacordado_delta_2_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 3) over ventana_6 as cdescubierto_preacordado_delta_3_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 4) over ventana_6 as cdescubierto_preacordado_delta_4_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 5) over ventana_6 as cdescubierto_preacordado_delta_5_lag, cdescubierto_preacordado - lag(cdescubierto_preacordado, 6) over ventana_6 as cdescubierto_preacordado_delta_6_lag
, regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_6 as cdescubierto_preacordado_slope_6, avg(cdescubierto_preacordado) over ventana_6 as cdescubierto_preacordado_media_6, lag(mcuenta_corriente, 1) over ventana_6 as mcuenta_corriente_lag_1, lag(mcuenta_corriente, 2) over ventana_6 as mcuenta_corriente_lag_2, lag(mcuenta_corriente, 3) over ventana_6 as mcuenta_corriente_lag_3, lag(mcuenta_corriente, 4) over ventana_6 as mcuenta_corriente_lag_4, lag(mcuenta_corriente, 5) over ventana_6 as mcuenta_corriente_lag_5, lag(mcuenta_corriente, 6) over ventana_6 as mcuenta_corriente_lag_6, mcuenta_corriente - lag(mcuenta_corriente, 1) over ventana_6 as mcuenta_corriente_delta_1_lag, mcuenta_corriente - lag(mcuenta_corriente, 2) over ventana_6 as mcuenta_corriente_delta_2_lag, mcuenta_corriente - lag(mcuenta_corriente, 3) over ventana_6 as mcuenta_corriente_delta_3_lag, mcuenta_corriente - lag(mcuenta_corriente, 4) over ventana_6 as mcuenta_corriente_delta_4_lag, mcuenta_corriente - lag(mcuenta_corriente, 5) over ventana_6 as mcuenta_corriente_delta_5_lag, mcuenta_corriente - lag(mcuenta_corriente, 6) over ventana_6 as mcuenta_corriente_delta_6_lag
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as mcuenta_corriente_slope_6, avg(mcuenta_corriente) over ventana_6 as mcuenta_corriente_media_6, lag(visa_status, 1) over ventana_6 as visa_status_lag_1, lag(visa_status, 2) over ventana_6 as visa_status_lag_2, lag(visa_status, 3) over ventana_6 as visa_status_lag_3, lag(visa_status, 4) over ventana_6 as visa_status_lag_4, lag(visa_status, 5) over ventana_6 as visa_status_lag_5, lag(visa_status, 6) over ventana_6 as visa_status_lag_6, visa_status - lag(visa_status, 1) over ventana_6 as visa_status_delta_1_lag, visa_status - lag(visa_status, 2) over ventana_6 as visa_status_delta_2_lag, visa_status - lag(visa_status, 3) over ventana_6 as visa_status_delta_3_lag, visa_status - lag(visa_status, 4) over ventana_6 as visa_status_delta_4_lag, visa_status - lag(visa_status, 5) over ventana_6 as visa_status_delta_5_lag, visa_status - lag(visa_status, 6) over ventana_6 as visa_status_delta_6_lag
, regr_slope(visa_status, cliente_antiguedad) over ventana_6 as visa_status_slope_6, avg(visa_status) over ventana_6 as visa_status_media_6, lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_lag_1, lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_lag_2, lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_lag_3, lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_lag_4, lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_lag_5, lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_lag_6, ctrx_quarter - lag(ctrx_quarter, 1) over ventana_6 as ctrx_quarter_delta_1_lag, ctrx_quarter - lag(ctrx_quarter, 2) over ventana_6 as ctrx_quarter_delta_2_lag, ctrx_quarter - lag(ctrx_quarter, 3) over ventana_6 as ctrx_quarter_delta_3_lag, ctrx_quarter - lag(ctrx_quarter, 4) over ventana_6 as ctrx_quarter_delta_4_lag, ctrx_quarter - lag(ctrx_quarter, 5) over ventana_6 as ctrx_quarter_delta_5_lag, ctrx_quarter - lag(ctrx_quarter, 6) over ventana_6 as ctrx_quarter_delta_6_lag
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6, avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6, lag(mpasivos_margen, 1) over ventana_6 as mpasivos_margen_lag_1, lag(mpasivos_margen, 2) over ventana_6 as mpasivos_margen_lag_2, lag(mpasivos_margen, 3) over ventana_6 as mpasivos_margen_lag_3, lag(mpasivos_margen, 4) over ventana_6 as mpasivos_margen_lag_4, lag(mpasivos_margen, 5) over ventana_6 as mpasivos_margen_lag_5, lag(mpasivos_margen, 6) over ventana_6 as mpasivos_margen_lag_6, mpasivos_margen - lag(mpasivos_margen, 1) over ventana_6 as mpasivos_margen_delta_1_lag, mpasivos_margen - lag(mpasivos_margen, 2) over ventana_6 as mpasivos_margen_delta_2_lag, mpasivos_margen - lag(mpasivos_margen, 3) over ventana_6 as mpasivos_margen_delta_3_lag, mpasivos_margen - lag(mpasivos_margen, 4) over ventana_6 as mpasivos_margen_delta_4_lag, mpasivos_margen - lag(mpasivos_margen, 5) over ventana_6 as mpasivos_margen_delta_5_lag, mpasivos_margen - lag(mpasivos_margen, 6) over ventana_6 as mpasivos_margen_delta_6_lag
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as mpasivos_margen_slope_6, avg(mpasivos_margen) over ventana_6 as mpasivos_margen_media_6, lag(mautoservicio, 1) over ventana_6 as mautoservicio_lag_1, lag(mautoservicio, 2) over ventana_6 as mautoservicio_lag_2, lag(mautoservicio, 3) over ventana_6 as mautoservicio_lag_3, lag(mautoservicio, 4) over ventana_6 as mautoservicio_lag_4, lag(mautoservicio, 5) over ventana_6 as mautoservicio_lag_5, lag(mautoservicio, 6) over ventana_6 as mautoservicio_lag_6, mautoservicio - lag(mautoservicio, 1) over ventana_6 as mautoservicio_delta_1_lag, mautoservicio - lag(mautoservicio, 2) over ventana_6 as mautoservicio_delta_2_lag, mautoservicio - lag(mautoservicio, 3) over ventana_6 as mautoservicio_delta_3_lag, mautoservicio - lag(mautoservicio, 4) over ventana_6 as mautoservicio_delta_4_lag, mautoservicio - lag(mautoservicio, 5) over ventana_6 as mautoservicio_delta_5_lag, mautoservicio - lag(mautoservicio, 6) over ventana_6 as mautoservicio_delta_6_lag
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as mautoservicio_slope_6, avg(mautoservicio) over ventana_6 as mautoservicio_media_6, lag(ctarjeta_debito_transacciones, 1) over ventana_6 as ctarjeta_debito_transacciones_lag_1, lag(ctarjeta_debito_transacciones, 2) over ventana_6 as ctarjeta_debito_transacciones_lag_2, lag(ctarjeta_debito_transacciones, 3) over ventana_6 as ctarjeta_debito_transacciones_lag_3, lag(ctarjeta_debito_transacciones, 4) over ventana_6 as ctarjeta_debito_transacciones_lag_4, lag(ctarjeta_debito_transacciones, 5) over ventana_6 as ctarjeta_debito_transacciones_lag_5, lag(ctarjeta_debito_transacciones, 6) over ventana_6 as ctarjeta_debito_transacciones_lag_6, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 1) over ventana_6 as ctarjeta_debito_transacciones_delta_1_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 2) over ventana_6 as ctarjeta_debito_transacciones_delta_2_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 3) over ventana_6 as ctarjeta_debito_transacciones_delta_3_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 4) over ventana_6 as ctarjeta_debito_transacciones_delta_4_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 5) over ventana_6 as ctarjeta_debito_transacciones_delta_5_lag, ctarjeta_debito_transacciones - lag(ctarjeta_debito_transacciones, 6) over ventana_6 as ctarjeta_debito_transacciones_delta_6_lag
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_debito_transacciones_slope_6, avg(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_transacciones_media_6, lag(mtransferencias_emitidas, 1) over ventana_6 as mtransferencias_emitidas_lag_1, lag(mtransferencias_emitidas, 2) over ventana_6 as mtransferencias_emitidas_lag_2, lag(mtransferencias_emitidas, 3) over ventana_6 as mtransferencias_emitidas_lag_3, lag(mtransferencias_emitidas, 4) over ventana_6 as mtransferencias_emitidas_lag_4, lag(mtransferencias_emitidas, 5) over ventana_6 as mtransferencias_emitidas_lag_5, lag(mtransferencias_emitidas, 6) over ventana_6 as mtransferencias_emitidas_lag_6, mtransferencias_emitidas - lag(mtransferencias_emitidas, 1) over ventana_6 as mtransferencias_emitidas_delta_1_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 2) over ventana_6 as mtransferencias_emitidas_delta_2_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 3) over ventana_6 as mtransferencias_emitidas_delta_3_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 4) over ventana_6 as mtransferencias_emitidas_delta_4_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 5) over ventana_6 as mtransferencias_emitidas_delta_5_lag, mtransferencias_emitidas - lag(mtransferencias_emitidas, 6) over ventana_6 as mtransferencias_emitidas_delta_6_lag
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_6 as mtransferencias_emitidas_slope_6, avg(mtransferencias_emitidas) over ventana_6 as mtransferencias_emitidas_media_6, lag(ctransferencias_emitidas, 1) over ventana_6 as ctransferencias_emitidas_lag_1, lag(ctransferencias_emitidas, 2) over ventana_6 as ctransferencias_emitidas_lag_2, lag(ctransferencias_emitidas, 3) over ventana_6 as ctransferencias_emitidas_lag_3, lag(ctransferencias_emitidas, 4) over ventana_6 as ctransferencias_emitidas_lag_4, lag(ctransferencias_emitidas, 5) over ventana_6 as ctransferencias_emitidas_lag_5, lag(ctransferencias_emitidas, 6) over ventana_6 as ctransferencias_emitidas_lag_6, ctransferencias_emitidas - lag(ctransferencias_emitidas, 1) over ventana_6 as ctransferencias_emitidas_delta_1_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 2) over ventana_6 as ctransferencias_emitidas_delta_2_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 3) over ventana_6 as ctransferencias_emitidas_delta_3_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 4) over ventana_6 as ctransferencias_emitidas_delta_4_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 5) over ventana_6 as ctransferencias_emitidas_delta_5_lag, ctransferencias_emitidas - lag(ctransferencias_emitidas, 6) over ventana_6 as ctransferencias_emitidas_delta_6_lag
, regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctransferencias_emitidas_slope_6, avg(ctransferencias_emitidas) over ventana_6 as ctransferencias_emitidas_media_6, lag(mpayroll, 1) over ventana_6 as mpayroll_lag_1, lag(mpayroll, 2) over ventana_6 as mpayroll_lag_2, lag(mpayroll, 3) over ventana_6 as mpayroll_lag_3, lag(mpayroll, 4) over ventana_6 as mpayroll_lag_4, lag(mpayroll, 5) over ventana_6 as mpayroll_lag_5, lag(mpayroll, 6) over ventana_6 as mpayroll_lag_6, mpayroll - lag(mpayroll, 1) over ventana_6 as mpayroll_delta_1_lag, mpayroll - lag(mpayroll, 2) over ventana_6 as mpayroll_delta_2_lag, mpayroll - lag(mpayroll, 3) over ventana_6 as mpayroll_delta_3_lag, mpayroll - lag(mpayroll, 4) over ventana_6 as mpayroll_delta_4_lag, mpayroll - lag(mpayroll, 5) over ventana_6 as mpayroll_delta_5_lag, mpayroll - lag(mpayroll, 6) over ventana_6 as mpayroll_delta_6_lag
, regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as mpayroll_slope_6, avg(mpayroll) over ventana_6 as mpayroll_media_6, lag(cpayroll_trx, 1) over ventana_6 as cpayroll_trx_lag_1, lag(cpayroll_trx, 2) over ventana_6 as cpayroll_trx_lag_2, lag(cpayroll_trx, 3) over ventana_6 as cpayroll_trx_lag_3, lag(cpayroll_trx, 4) over ventana_6 as cpayroll_trx_lag_4, lag(cpayroll_trx, 5) over ventana_6 as cpayroll_trx_lag_5, lag(cpayroll_trx, 6) over ventana_6 as cpayroll_trx_lag_6, cpayroll_trx - lag(cpayroll_trx, 1) over ventana_6 as cpayroll_trx_delta_1_lag, cpayroll_trx - lag(cpayroll_trx, 2) over ventana_6 as cpayroll_trx_delta_2_lag, cpayroll_trx - lag(cpayroll_trx, 3) over ventana_6 as cpayroll_trx_delta_3_lag, cpayroll_trx - lag(cpayroll_trx, 4) over ventana_6 as cpayroll_trx_delta_4_lag, cpayroll_trx - lag(cpayroll_trx, 5) over ventana_6 as cpayroll_trx_delta_5_lag, cpayroll_trx - lag(cpayroll_trx, 6) over ventana_6 as cpayroll_trx_delta_6_lag
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as cpayroll_trx_slope_6, avg(cpayroll_trx) over ventana_6 as cpayroll_trx_media_6, lag(mcomisiones_mantenimiento, 1) over ventana_6 as mcomisiones_mantenimiento_lag_1, lag(mcomisiones_mantenimiento, 2) over ventana_6 as mcomisiones_mantenimiento_lag_2, lag(mcomisiones_mantenimiento, 3) over ventana_6 as mcomisiones_mantenimiento_lag_3, lag(mcomisiones_mantenimiento, 4) over ventana_6 as mcomisiones_mantenimiento_lag_4, lag(mcomisiones_mantenimiento, 5) over ventana_6 as mcomisiones_mantenimiento_lag_5, lag(mcomisiones_mantenimiento, 6) over ventana_6 as mcomisiones_mantenimiento_lag_6, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 1) over ventana_6 as mcomisiones_mantenimiento_delta_1_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 2) over ventana_6 as mcomisiones_mantenimiento_delta_2_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 3) over ventana_6 as mcomisiones_mantenimiento_delta_3_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 4) over ventana_6 as mcomisiones_mantenimiento_delta_4_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 5) over ventana_6 as mcomisiones_mantenimiento_delta_5_lag, mcomisiones_mantenimiento - lag(mcomisiones_mantenimiento, 6) over ventana_6 as mcomisiones_mantenimiento_delta_6_lag
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as mcomisiones_mantenimiento_slope_6, avg(mcomisiones_mantenimiento) over ventana_6 as mcomisiones_mantenimiento_media_6, lag(mcomisiones_otras, 1) over ventana_6 as mcomisiones_otras_lag_1, lag(mcomisiones_otras, 2) over ventana_6 as mcomisiones_otras_lag_2, lag(mcomisiones_otras, 3) over ventana_6 as mcomisiones_otras_lag_3, lag(mcomisiones_otras, 4) over ventana_6 as mcomisiones_otras_lag_4, lag(mcomisiones_otras, 5) over ventana_6 as mcomisiones_otras_lag_5, lag(mcomisiones_otras, 6) over ventana_6 as mcomisiones_otras_lag_6, mcomisiones_otras - lag(mcomisiones_otras, 1) over ventana_6 as mcomisiones_otras_delta_1_lag, mcomisiones_otras - lag(mcomisiones_otras, 2) over ventana_6 as mcomisiones_otras_delta_2_lag, mcomisiones_otras - lag(mcomisiones_otras, 3) over ventana_6 as mcomisiones_otras_delta_3_lag, mcomisiones_otras - lag(mcomisiones_otras, 4) over ventana_6 as mcomisiones_otras_delta_4_lag, mcomisiones_otras - lag(mcomisiones_otras, 5) over ventana_6 as mcomisiones_otras_delta_5_lag, mcomisiones_otras - lag(mcomisiones_otras, 6) over ventana_6 as mcomisiones_otras_delta_6_lag
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as mcomisiones_otras_slope_6, avg(mcomisiones_otras) over ventana_6 as mcomisiones_otras_media_6, lag(mcomisiones, 1) over ventana_6 as mcomisiones_lag_1, lag(mcomisiones, 2) over ventana_6 as mcomisiones_lag_2, lag(mcomisiones, 3) over ventana_6 as mcomisiones_lag_3, lag(mcomisiones, 4) over ventana_6 as mcomisiones_lag_4, lag(mcomisiones, 5) over ventana_6 as mcomisiones_lag_5, lag(mcomisiones, 6) over ventana_6 as mcomisiones_lag_6, mcomisiones - lag(mcomisiones, 1) over ventana_6 as mcomisiones_delta_1_lag, mcomisiones - lag(mcomisiones, 2) over ventana_6 as mcomisiones_delta_2_lag, mcomisiones - lag(mcomisiones, 3) over ventana_6 as mcomisiones_delta_3_lag, mcomisiones - lag(mcomisiones, 4) over ventana_6 as mcomisiones_delta_4_lag, mcomisiones - lag(mcomisiones, 5) over ventana_6 as mcomisiones_delta_5_lag, mcomisiones - lag(mcomisiones, 6) over ventana_6 as mcomisiones_delta_6_lag
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as mcomisiones_slope_6, avg(mcomisiones) over ventana_6 as mcomisiones_media_6, lag(mcaja_ahorro, 1) over ventana_6 as mcaja_ahorro_lag_1, lag(mcaja_ahorro, 2) over ventana_6 as mcaja_ahorro_lag_2, lag(mcaja_ahorro, 3) over ventana_6 as mcaja_ahorro_lag_3, lag(mcaja_ahorro, 4) over ventana_6 as mcaja_ahorro_lag_4, lag(mcaja_ahorro, 5) over ventana_6 as mcaja_ahorro_lag_5, lag(mcaja_ahorro, 6) over ventana_6 as mcaja_ahorro_lag_6, mcaja_ahorro - lag(mcaja_ahorro, 1) over ventana_6 as mcaja_ahorro_delta_1_lag, mcaja_ahorro - lag(mcaja_ahorro, 2) over ventana_6 as mcaja_ahorro_delta_2_lag, mcaja_ahorro - lag(mcaja_ahorro, 3) over ventana_6 as mcaja_ahorro_delta_3_lag, mcaja_ahorro - lag(mcaja_ahorro, 4) over ventana_6 as mcaja_ahorro_delta_4_lag, mcaja_ahorro - lag(mcaja_ahorro, 5) over ventana_6 as mcaja_ahorro_delta_5_lag, mcaja_ahorro - lag(mcaja_ahorro, 6) over ventana_6 as mcaja_ahorro_delta_6_lag
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as mcaja_ahorro_slope_6, avg(mcaja_ahorro) over ventana_6 as mcaja_ahorro_media_6, lag(mcuentas_saldo, 1) over ventana_6 as mcuentas_saldo_lag_1, lag(mcuentas_saldo, 2) over ventana_6 as mcuentas_saldo_lag_2, lag(mcuentas_saldo, 3) over ventana_6 as mcuentas_saldo_lag_3, lag(mcuentas_saldo, 4) over ventana_6 as mcuentas_saldo_lag_4, lag(mcuentas_saldo, 5) over ventana_6 as mcuentas_saldo_lag_5, lag(mcuentas_saldo, 6) over ventana_6 as mcuentas_saldo_lag_6, mcuentas_saldo - lag(mcuentas_saldo, 1) over ventana_6 as mcuentas_saldo_delta_1_lag, mcuentas_saldo - lag(mcuentas_saldo, 2) over ventana_6 as mcuentas_saldo_delta_2_lag, mcuentas_saldo - lag(mcuentas_saldo, 3) over ventana_6 as mcuentas_saldo_delta_3_lag, mcuentas_saldo - lag(mcuentas_saldo, 4) over ventana_6 as mcuentas_saldo_delta_4_lag, mcuentas_saldo - lag(mcuentas_saldo, 5) over ventana_6 as mcuentas_saldo_delta_5_lag, mcuentas_saldo - lag(mcuentas_saldo, 6) over ventana_6 as mcuentas_saldo_delta_6_lag
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as mcuentas_saldo_slope_6, avg(mcuentas_saldo) over ventana_6 as mcuentas_saldo_media_6, lag(mtarjeta_visa_consumo, 1) over ventana_6 as mtarjeta_visa_consumo_lag_1, lag(mtarjeta_visa_consumo, 2) over ventana_6 as mtarjeta_visa_consumo_lag_2, lag(mtarjeta_visa_consumo, 3) over ventana_6 as mtarjeta_visa_consumo_lag_3, lag(mtarjeta_visa_consumo, 4) over ventana_6 as mtarjeta_visa_consumo_lag_4, lag(mtarjeta_visa_consumo, 5) over ventana_6 as mtarjeta_visa_consumo_lag_5, lag(mtarjeta_visa_consumo, 6) over ventana_6 as mtarjeta_visa_consumo_lag_6, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 1) over ventana_6 as mtarjeta_visa_consumo_delta_1_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 2) over ventana_6 as mtarjeta_visa_consumo_delta_2_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 3) over ventana_6 as mtarjeta_visa_consumo_delta_3_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 4) over ventana_6 as mtarjeta_visa_consumo_delta_4_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 5) over ventana_6 as mtarjeta_visa_consumo_delta_5_lag, mtarjeta_visa_consumo - lag(mtarjeta_visa_consumo, 6) over ventana_6 as mtarjeta_visa_consumo_delta_6_lag
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as mtarjeta_visa_consumo_slope_6, avg(mtarjeta_visa_consumo) over ventana_6 as mtarjeta_visa_consumo_media_6, lag(mprestamos_personales, 1) over ventana_6 as mprestamos_personales_lag_1, lag(mprestamos_personales, 2) over ventana_6 as mprestamos_personales_lag_2, lag(mprestamos_personales, 3) over ventana_6 as mprestamos_personales_lag_3, lag(mprestamos_personales, 4) over ventana_6 as mprestamos_personales_lag_4, lag(mprestamos_personales, 5) over ventana_6 as mprestamos_personales_lag_5, lag(mprestamos_personales, 6) over ventana_6 as mprestamos_personales_lag_6, mprestamos_personales - lag(mprestamos_personales, 1) over ventana_6 as mprestamos_personales_delta_1_lag, mprestamos_personales - lag(mprestamos_personales, 2) over ventana_6 as mprestamos_personales_delta_2_lag, mprestamos_personales - lag(mprestamos_personales, 3) over ventana_6 as mprestamos_personales_delta_3_lag, mprestamos_personales - lag(mprestamos_personales, 4) over ventana_6 as mprestamos_personales_delta_4_lag, mprestamos_personales - lag(mprestamos_personales, 5) over ventana_6 as mprestamos_personales_delta_5_lag, mprestamos_personales - lag(mprestamos_personales, 6) over ventana_6 as mprestamos_personales_delta_6_lag
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as mprestamos_personales_slope_6, avg(mprestamos_personales) over ventana_6 as mprestamos_personales_media_6
, regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_3 as cdescubierto_preacordado_slope_3, avg(cdescubierto_preacordado) over ventana_3 as cdescubierto_preacordado_media_3
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as mcuenta_corriente_slope_3, avg(mcuenta_corriente) over ventana_3 as mcuenta_corriente_media_3
, regr_slope(visa_status, cliente_antiguedad) over ventana_3 as visa_status_slope_3, avg(visa_status) over ventana_3 as visa_status_media_3
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3, avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3, avg(mpasivos_margen) over ventana_3 as mpasivos_margen_media_3
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_3 as mautoservicio_slope_3, avg(mautoservicio) over ventana_3 as mautoservicio_media_3
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_3 as ctarjeta_debito_transacciones_slope_3, avg(ctarjeta_debito_transacciones) over ventana_3 as ctarjeta_debito_transacciones_media_3
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_3 as mtransferencias_emitidas_slope_3, avg(mtransferencias_emitidas) over ventana_3 as mtransferencias_emitidas_media_3
, regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_3 as ctransferencias_emitidas_slope_3, avg(ctransferencias_emitidas) over ventana_3 as ctransferencias_emitidas_media_3
, regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3, avg(mpayroll) over ventana_3 as mpayroll_media_3
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3, avg(cpayroll_trx) over ventana_3 as cpayroll_trx_media_3
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mcomisiones_mantenimiento_slope_3, avg(mcomisiones_mantenimiento) over ventana_3 as mcomisiones_mantenimiento_media_3
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_3 as mcomisiones_otras_slope_3, avg(mcomisiones_otras) over ventana_3 as mcomisiones_otras_media_3
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_3 as mcomisiones_slope_3, avg(mcomisiones) over ventana_3 as mcomisiones_media_3
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3, avg(mcaja_ahorro) over ventana_3 as mcaja_ahorro_media_3
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3, avg(mcuentas_saldo) over ventana_3 as mcuentas_saldo_media_3
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as mtarjeta_visa_consumo_slope_3, avg(mtarjeta_visa_consumo) over ventana_3 as mtarjeta_visa_consumo_media_3
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3, avg(mprestamos_personales) over ventana_3 as mprestamos_personales_media_3
, regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_3 as cdescubierto_preacordado_slope_3, avg(cdescubierto_preacordado) over ventana_3 as cdescubierto_preacordado_media_3
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as mcuenta_corriente_slope_3, avg(mcuenta_corriente) over ventana_3 as mcuenta_corriente_media_3
, regr_slope(visa_status, cliente_antiguedad) over ventana_3 as visa_status_slope_3, avg(visa_status) over ventana_3 as visa_status_media_3
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3, avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3, avg(mpasivos_margen) over ventana_3 as mpasivos_margen_media_3
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_3 as mautoservicio_slope_3, avg(mautoservicio) over ventana_3 as mautoservicio_media_3
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_3 as ctarjeta_debito_transacciones_slope_3, avg(ctarjeta_debito_transacciones) over ventana_3 as ctarjeta_debito_transacciones_media_3
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_3 as mtransferencias_emitidas_slope_3, avg(mtransferencias_emitidas) over ventana_3 as mtransferencias_emitidas_media_3
, regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_3 as ctransferencias_emitidas_slope_3, avg(ctransferencias_emitidas) over ventana_3 as ctransferencias_emitidas_media_3
, regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3, avg(mpayroll) over ventana_3 as mpayroll_media_3
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3, avg(cpayroll_trx) over ventana_3 as cpayroll_trx_media_3
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mcomisiones_mantenimiento_slope_3, avg(mcomisiones_mantenimiento) over ventana_3 as mcomisiones_mantenimiento_media_3
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_3 as mcomisiones_otras_slope_3, avg(mcomisiones_otras) over ventana_3 as mcomisiones_otras_media_3
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_3 as mcomisiones_slope_3, avg(mcomisiones) over ventana_3 as mcomisiones_media_3
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3, avg(mcaja_ahorro) over ventana_3 as mcaja_ahorro_media_3
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3, avg(mcuentas_saldo) over ventana_3 as mcuentas_saldo_media_3
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as mtarjeta_visa_consumo_slope_3, avg(mtarjeta_visa_consumo) over ventana_3 as mtarjeta_visa_consumo_media_3
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3, avg(mprestamos_personales) over ventana_3 as mprestamos_personales_media_3
FROM competencia_02
WINDOW ventana_3 AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
WINDOW ventana_6 AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 5 PRECEDING AND CURRENT ROW)
WINDOW ventana_9 AS (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 8 PRECEDING AND CURRENT ROW)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4min 41s, sys: 41.5 s, total: 5min 23s
Wall time: 3min 39s


,Success


In [ ]:
#guardo el archivo en el bucket

%%time
%%sql
copy competencia_02 to 'competencia_02_FEH_2.csv.gz' (FORMAT CSV, HEADER)

In [ ]:
#verifico el archivo

%%time
%%sql
create or replace table competencia_02_verificar as
select
  numero_de_cliente
  , foto_mes
from read_csv_auto('competencia_02_FEH_2.csv.gz')

CPU times: user 240 ms, sys: 18.7 ms, total: 259 ms
Wall time: 355 ms


,Success


In [ ]:
%%time
%sql SELECT count(*) FROM competencia_02_verificar

CPU times: user 138 ms, sys: 5.91 ms, total: 144 ms
Wall time: 224 ms


,count_star()
0,31
